In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2 # Check your ros path, if opencv can not import, edit your bashrc file.
import time

Configure depth and color streams.

In [2]:
pipeline = rs.pipeline()
config = rs.config()

Configure the stream format, size, fps.

In [3]:
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)

In [4]:
# Start streaming.
pipeline.start(config)

In [5]:
# Wait for a coherent pair of frames: depth and color. Default is 5000 ms == 5s.
frames = pipeline.wait_for_frames()
#?pipeline.wait_for_frames

In [6]:
depth_frame = frames.get_depth_frame()

In [7]:
color_frame = frames.get_color_frame()

In [8]:
depth_image = np.asanyarray(depth_frame.get_data())
depth_data = depth_frame.as_frame().get_data()

'''
# As same as up.
image = np.asanyarray(depth_data)
print(image)
print(depth_image)
'''

depth_image.shape

(480, 640)

In [9]:
color_image = np.asanyarray(color_frame.get_data())
color_image.shape

(480, 640, 3)

Apply colormap on depth image(image must be converted to 8-bit per pixel first).

In [10]:
depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
depth_colormap

array([[[128,   0,   0],
        [128,   0,   0],
        [128,   0,   0],
        ...,
        [128,   0,   0],
        [128,   0,   0],
        [128,   0,   0]],

       [[128,   0,   0],
        [128,   0,   0],
        [128,   0,   0],
        ...,
        [128,   0,   0],
        [128,   0,   0],
        [128,   0,   0]],

       [[128,   0,   0],
        [128,   0,   0],
        [128,   0,   0],
        ...,
        [128,   0,   0],
        [128,   0,   0],
        [128,   0,   0]],

       ...,

       [[128,   0,   0],
        [128,   0,   0],
        [128,   0,   0],
        ...,
        [128,   0,   0],
        [128,   0,   0],
        [128,   0,   0]],

       [[128,   0,   0],
        [128,   0,   0],
        [128,   0,   0],
        ...,
        [128,   0,   0],
        [128,   0,   0],
        [128,   0,   0]],

       [[128,   0,   0],
        [128,   0,   0],
        [128,   0,   0],
        ...,
        [128,   0,   0],
        [128,   0,   0],
        [128,   0,   0]]

Stack both images horizontally.

In [11]:
images = np.hstack((color_image, depth_colormap))

Show Images.

In [12]:
'''
cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
cv2.imshow('RealSense', images)
cv2.waitKey(1)
'''

"\ncv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)\ncv2.imshow('RealSense', images)\ncv2.waitKey(1)\n"

## Loop get the frame.

In [13]:
try:
    while True:
        
        frames = pipeline.wait_for_frames()
        depth_frame = frames.get_depth_frame()
        color_frame = frames.get_color_frame()
        if not depth_frame or not color_frame:
            continue
        
        # Convert images to numpy arrays
        depth_image = np.asanyarray(depth_frame.get_data())
        color_image = np.asanyarray(color_frame.get_data())
        
        # Apply colormap on depth image (image must be converted to 8-bit per pixel first).
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_HSV)
        
        # Stack both images horizontally
        images = np.hstack((color_image, depth_colormap))
        
        # Show images
        cv2.namedWindow('RealSense', cv2.WINDOW_AUTOSIZE)
        cv2.imshow('RealSense', images)
        #cv2.imwrite("/home/commaai-03/image/color/%.9f.jpg" % time.time(), color_image)
        if cv2.waitKey(1) == ord('q'):
            cv2.destroyAllWindows()
            break
        
finally:
    # pipeline.stop()
    pass

In [ ]:
# If you wanna close the device. Run this.
pipeline.stop